# parameter tuning
## 1.Try different parameter on basic network structure， keep the best parameter
## 2. IF a word has a UpperCase character , it's very likely a skill word. 
##     Add a Case_sensitive layer to the network, test the accuracy

### import NER dataset

In [1]:
import pandas as pd

In [2]:
DF = pd.read_excel( r'E:\dataset\NER_total.xls' )
DF.sample(10)

,Sentence,Output
605,"['5', '+', 'years', 'of', 'SAS', 'experience']","['O', 'O', 'O', 'O', 'B', 'O']"
636,"['(', 'e.g.', 'Python', ',', 'Ruby', ',', 'Jav...","['O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', ..."
462,"['Azure', '/']","['B', 'O']"
534,"['Knowledge', 'of', 'digital', 'marketing', 'a...","['O', 'O', 'B', 'I', 'O', 'O', 'O']"
229,"['Deep', 'Learning', ',']","['B', 'I', 'O']"
840,"['specifically', 'Deep', 'Learning', ',', 'NLP...","['O', 'B', 'I', 'O', 'B', 'O', 'O', 'O', 'O', ..."
562,"['Proven', 'Experience', 'using', 'Python', ',...","['O', 'O', 'O', 'B', 'O', 'B', 'O', 'B']"
381,"['.', 'with', 'scom', '•', 'notifications', 's...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
751,"['Advanced', 'knowledge', 'of', 'database', 'd...","['O', 'O', 'O', 'B', 'I', 'O', 'O']"
54,"['Experience', 'in', 'machine', 'learning', 'a...","['O', 'O', 'B', 'I', 'O', 'B', 'I', 'O']"


### sample data

In [3]:
import random

ranIndex = random.randint( 0,len(DF) )

print('Sentence:')
print( DF.iloc[ranIndex]['Sentence'] )
print('Label')
print( DF.iloc[ranIndex]['Output'] )
print()

ranIndex = random.randint( 0,len(DF) )

print('Sentence:')
print( DF.iloc[ranIndex]['Sentence'] )
print('Label')
print( DF.iloc[ranIndex]['Output'] )
print()

ranIndex = random.randint( 0,len(DF) )

print('Sentence:')
print( DF.iloc[ranIndex]['Sentence'] )
print('Label')
print( DF.iloc[ranIndex]['Output'] )

Sentence:
['Proven', 'influencing', ',', 'communication', 'and', 'consulting', 'skills']
Label
['O', 'O', 'O', 'B', 'O', 'B', 'O']

Sentence:
['*', 'Proficiency', 'with', 'python', 'and', 'R']
Label
['O', 'O', 'O', 'B', 'O', 'B']

Sentence:
['(', 'e.g.', 'Python', ',', 'Ruby', ',', 'Java', ',', 'Scala', ')']
Label
['O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O']


In [4]:
sentence_list = DF['Sentence'].apply(eval)
label_list = DF['Output'].apply(eval)

sentence_list = [ w for w in sentence_list if len(w) <= 120]
label_list = [ w for w in label_list if len(w) <= 120]

# dataset split

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( sentence_list, label_list, test_size=0.2, random_state=42)

# PreProcess

In [6]:
import tensorflow as tf
from pprint import pprint

In [7]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
max_seq_length = 128

In [9]:
label = {}
_label = {}


label_path = "E://bert-model//tag_dict.txt"
f_label = open(label_path, 'r+', encoding='utf-8')
for line in f_label:
    content = line.strip().split()
    label[content[0].strip()] = content[1].strip()
    _label[content[1].strip()] = content[0].strip()
    
    
vocab = {}
vocab_path = r"E:\Project\uncased_L-2_H-128_A-2\vocab.txt"
with open(vocab_path, 'r+', encoding='utf-8') as f_vocab:
    for line in f_vocab.readlines():
        vocab[line.strip()] = len(vocab)

In [10]:
def PreProcessInputData( text ):
    word_labels = []
    seq_types = []

    for sequence in text:
        len_text = len(sequence)

        ###########################################
        temp_word_labels = []
        temp_word_labels.append( 101 )            
        for w in sequence:
            temp_word_labels.append( vocab.get(str(w).lower(),100) )
        temp_word_labels.append( 102 )

        ###########################################
        ###########################################
        temp_seq_types = [1] * len(temp_word_labels) +  [0] * ( max_seq_length - len( temp_word_labels ))
        temp_word_labels = temp_word_labels + [0] * ( max_seq_length - len( temp_word_labels ))

        word_labels.append( temp_word_labels )
        seq_types.append( temp_seq_types )

    return word_labels, seq_types

In [11]:
def PreProcessOutputData( text ):
    tags = []
    for line in text:
        tag = [0]
        for item in line:
            tag.append( int(label[item.strip()]) )
        tag.append(0)
        tags.append(tag)

    pad_tags = pad_sequences(tags, maxlen=max_seq_length, padding="post", truncating="post")
    result_tags = np.expand_dims(pad_tags, 2)
    return result_tags

# Data PreProcessing

In [12]:
train_sentence_list_word_labels ,train_sentence_list_seq_types = PreProcessInputData( X_train )
train_output_label_list = PreProcessOutputData( y_train )

In [13]:
test_sentence_list_word_labels ,test_sentence_list_seq_types = PreProcessInputData( X_test )
test_output_label_list = PreProcessOutputData( y_test )

# basic network structure define

# define_Layer

In [14]:
def define_Layer( layer_dict, my_input ):
    
    if layer_dict['layer_type'] == 'Bidirectional_LSTM':
        return Bidirectional( LSTM( layer_dict['lstmDim'],
                                         return_sequences = True,
                                         activation = layer_dict['activation'],
                                         dropout = layer_dict['dropout'],
                                         recurrent_dropout = layer_dict['dropout']) )(my_input)
    
    elif layer_dict['layer_type'] == 'Dense':
        return Dense(units = layer_dict['units'] , activation = layer_dict['activation'] )(my_input)
    
    elif layer_dict['layer_type'] == 'Dropout':
        return Dropout(dropout = layer_dict['dropout'] )(my_input)

# define network structure

In [15]:
def Train_Model( network_struct_dict ):
    print( '■' * 60 )
    print('network struct:')
    pprint( network_struct_dict )
    
    #################################################################
    ## use BERT embedding layer
    model_path = r"E:/Project/uncased_L-2_H-128_A-2/"
    bert = load_trained_model_from_checkpoint(
        model_path + "bert_config.json",
        model_path + "bert_model.ckpt",
        seq_len = max_seq_length
        )

    #make bert layer trainable
    for layer in bert.layers:
        layer.trainable = network_struct_dict['is_embeding_trainable']
        
    x1 = Input(shape=(None,))
    x2 = Input(shape=(None,))
    bert_out = bert([x1, x2])
    
    my_output = bert_out
    #################################################################
    # use layer_dict_List to build model
    for layer_dict in network_struct_dict['layer_dict_List']:
        my_input  = my_output
        my_output = define_Layer( layer_dict, my_input )
    
    
    #################################################################
    ## CRF output
    crf_out = CRF(len(label), sparse_target=True)(my_output)
    model = Model([x1, x2], crf_out)
    model.summary()

    model.compile(
        optimizer=network_struct_dict['optimizer'],
        loss=crf_loss,
        metrics=[crf_accuracy]
    )
    
    #################################################################
    ## training
    model.fit(x=[train_sentence_list_word_labels, train_sentence_list_seq_types],
                       y = train_output_label_list,
                       batch_size = network_struct_dict['batch_size'],
                       epochs = network_struct_dict['epochs'],
                       validation_split=0.2,
                       callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001)],
                       verbose = 1,
                       class_weight = 'auto')
    
    print('evaluate :')
    res = model.evaluate(x=[test_sentence_list_word_labels, test_sentence_list_seq_types],
                       y=test_output_label_list,batch_size=128)
    
    print( res )
    return res[1]

# try different parameter and keep the best

In [23]:
network_struct_dict_List = [
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.3},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : False,
        'optimizer' : Adam(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : Adam(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'sigmoid', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.4},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'sigmoid', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 32 ,'activation':'sigmoid', 'dropout' :0.2},
        ]
    },
    
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 64,
        'epochs': 100,
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 64 ,'activation':'sigmoid'},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : Adam(1e-4),
        'batch_size' : 64,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 32 ,'activation':'sigmoid'},
        ]
    }
]

In [25]:
best_network_struct = None
best_accurac = 0

for network_struct_dict in network_struct_dict_List:
    if best_accurac < Train_Model( network_struct_dict ):
        best_accurac = Train_Model( network_struct_dict )
        best_network_struct = network_struct_dict
        
print('Best Network Struct:')
pprint(best_network_struct)

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
network struct:
{'batch_size': 128,
 'epochs': 100,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.3,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 256}],
 'optimizer': <keras.optimizers.RMSprop object at 0x0000000036A72198>}
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_10 (

Train on 544 samples, validate on 136 samples
Epoch 1/100
544/544 [==============================] - 57s 104ms/step - loss: 18.2692 - crf_accuracy: 0.4698 - val_loss: 17.2330 - val_crf_accuracy: 0.8265
Epoch 2/100
544/544 [==============================] - 52s 95ms/step - loss: 17.5981 - crf_accuracy: 0.8152 - val_loss: 17.1472 - val_crf_accuracy: 0.8265
Epoch 3/100
544/544 [==============================] - 52s 96ms/step - loss: 17.5072 - crf_accuracy: 0.8154 - val_loss: 17.1011 - val_crf_accuracy: 0.8265
Epoch 4/100
544/544 [==============================] - 52s 96ms/step - loss: 17.4541 - crf_accuracy: 0.8264 - val_loss: 17.0477 - val_crf_accuracy: 0.8330
Epoch 5/100
544/544 [==============================] - 53s 97ms/step - loss: 17.4027 - crf_accuracy: 0.8316 - val_loss: 16.9735 - val_crf_accuracy: 0.8635
Epoch 6/100
544/544 [==============================] - 53s 98ms/step - loss: 17.3597 - crf_accuracy: 0.8400 - val_loss: 16.9600 - val_crf_accuracy: 0.8511
Epoch 7/100
544/544 [==

Epoch 19/100
544/544 [==============================] - 54s 98ms/step - loss: 17.0512 - crf_accuracy: 0.9079 - val_loss: 16.7317 - val_crf_accuracy: 0.9028
Epoch 20/100
544/544 [==============================] - 52s 95ms/step - loss: 17.0452 - crf_accuracy: 0.9070 - val_loss: 16.7210 - val_crf_accuracy: 0.9121
Epoch 21/100
544/544 [==============================] - 52s 96ms/step - loss: 17.0194 - crf_accuracy: 0.9146 - val_loss: 16.7178 - val_crf_accuracy: 0.9152
Epoch 22/100
544/544 [==============================] - 54s 100ms/step - loss: 17.0131 - crf_accuracy: 0.9208 - val_loss: 16.7243 - val_crf_accuracy: 0.9112
Epoch 23/100
544/544 [==============================] - 52s 96ms/step - loss: 17.0159 - crf_accuracy: 0.9146 - val_loss: 16.7079 - val_crf_accuracy: 0.9158
Epoch 24/100
544/544 [==============================] - 53s 97ms/step - loss: 16.9983 - crf_accuracy: 0.9220 - val_loss: 16.7064 - val_crf_accuracy: 0.9219
Epoch 25/100
544/544 [==============================] - 52s 96m

Train on 544 samples, validate on 136 samples
Epoch 1/100
544/544 [==============================] - 63s 116ms/step - loss: 18.7574 - crf_accuracy: 0.0242 - val_loss: 18.1172 - val_crf_accuracy: 0.0243
Epoch 2/100
544/544 [==============================] - 53s 98ms/step - loss: 18.3127 - crf_accuracy: 0.2994 - val_loss: 17.7267 - val_crf_accuracy: 0.8255
Epoch 3/100
544/544 [==============================] - 53s 97ms/step - loss: 17.9826 - crf_accuracy: 0.8121 - val_loss: 17.4915 - val_crf_accuracy: 0.8262
Epoch 4/100
544/544 [==============================] - 54s 100ms/step - loss: 17.7976 - crf_accuracy: 0.8156 - val_loss: 17.3877 - val_crf_accuracy: 0.8262
Epoch 5/100
544/544 [==============================] - 52s 96ms/step - loss: 17.7074 - crf_accuracy: 0.8153 - val_loss: 17.3360 - val_crf_accuracy: 0.8262
Epoch 6/100
544/544 [==============================] - 53s 98ms/step - loss: 17.6594 - crf_accuracy: 0.8146 - val_loss: 17.2959 - val_crf_accuracy: 0.8262
Epoch 7/100
544/544 [=

Train on 544 samples, validate on 136 samples
Epoch 1/100
544/544 [==============================] - 73s 134ms/step - loss: 18.7328 - crf_accuracy: 0.2744 - val_loss: 17.8624 - val_crf_accuracy: 0.4466
Epoch 2/100
544/544 [==============================] - 59s 109ms/step - loss: 18.0303 - crf_accuracy: 0.5205 - val_loss: 17.3591 - val_crf_accuracy: 0.8265
Epoch 3/100
544/544 [==============================] - 61s 112ms/step - loss: 17.6394 - crf_accuracy: 0.8127 - val_loss: 17.1545 - val_crf_accuracy: 0.8262
Epoch 4/100
544/544 [==============================] - 60s 110ms/step - loss: 17.4722 - crf_accuracy: 0.8145 - val_loss: 17.0307 - val_crf_accuracy: 0.8303
Epoch 5/100
544/544 [==============================] - 60s 110ms/step - loss: 17.3588 - crf_accuracy: 0.8311 - val_loss: 16.9322 - val_crf_accuracy: 0.8748
Epoch 6/100
544/544 [==============================] - 60s 111ms/step - loss: 17.2652 - crf_accuracy: 0.8600 - val_loss: 16.8782 - val_crf_accuracy: 0.8617
Epoch 7/100
544/54

...

544/544 [==============================] - 223s 410ms/step - loss: 17.1685 - crf_accuracy: 0.8721 - val_loss: 16.7866 - val_crf_accuracy: 0.8916
Epoch 20/100
544/544 [==============================] - 234s 429ms/step - loss: 17.1292 - crf_accuracy: 0.8872 - val_loss: 16.7910 - val_crf_accuracy: 0.8852
Epoch 21/100
544/544 [==============================] - 229s 420ms/step - loss: 17.1068 - crf_accuracy: 0.8947 - val_loss: 16.7550 - val_crf_accuracy: 0.8948
Epoch 22/100
544/544 [==============================] - 234s 429ms/step - loss: 17.0830 - crf_accuracy: 0.8986 - val_loss: 16.7454 - val_crf_accuracy: 0.8980
Epoch 23/100
544/544 [==============================] - 239s 439ms/step - loss: 17.0805 - crf_accuracy: 0.9027 - val_loss: 16.7693 - val_crf_accuracy: 0.8953
Epoch 24/100
544/544 [==============================] - 238s 437ms/step - loss: 17.0593 - crf_accuracy: 0.9071 - val_loss: 16.7348 - val_crf_accuracy: 0.9038
Epoch 25/100
544/544 [==============================] - 245s 451m

544/544 [==============================] - 318s 584ms/step - loss: 17.2786 - crf_accuracy: 0.8255 - val_loss: 16.9138 - val_crf_accuracy: 0.8470
Epoch 17/100
544/544 [==============================] - 318s 584ms/step - loss: 17.2624 - crf_accuracy: 0.8268 - val_loss: 16.9552 - val_crf_accuracy: 0.8278
Epoch 18/100
544/544 [==============================] - 325s 597ms/step - loss: 17.2459 - crf_accuracy: 0.8383 - val_loss: 16.8690 - val_crf_accuracy: 0.8619
Epoch 19/100
544/544 [==============================] - 318s 584ms/step - loss: 17.2069 - crf_accuracy: 0.8523 - val_loss: 16.8679 - val_crf_accuracy: 0.8817
Epoch 20/100
544/544 [==============================] - 313s 576ms/step - loss: 17.2078 - crf_accuracy: 0.8509 - val_loss: 16.8259 - val_crf_accuracy: 0.8679
Epoch 21/100
544/544 [==============================] - 329s 604ms/step - loss: 17.1557 - crf_accuracy: 0.8679 - val_loss: 16.8006 - val_crf_accuracy: 0.8812
Epoch 22/100
544/544 [==============================] - 320s 588m

544/544 [==============================] - 229s 421ms/step - loss: 17.6140 - crf_accuracy: 0.8147 - val_loss: 17.2472 - val_crf_accuracy: 0.8262
Epoch 7/100
544/544 [==============================] - 230s 422ms/step - loss: 17.5947 - crf_accuracy: 0.8133 - val_loss: 17.2240 - val_crf_accuracy: 0.8177
Epoch 8/100
544/544 [==============================] - 230s 423ms/step - loss: 17.5909 - crf_accuracy: 0.8072 - val_loss: 17.2162 - val_crf_accuracy: 0.8262
Epoch 9/100
544/544 [==============================] - 248s 456ms/step - loss: 17.5738 - crf_accuracy: 0.8102 - val_loss: 17.2031 - val_crf_accuracy: 0.8262
Epoch 10/100
544/544 [==============================] - 269s 495ms/step - loss: 17.5620 - crf_accuracy: 0.8115 - val_loss: 17.2006 - val_crf_accuracy: 0.8262
Epoch 11/100
544/544 [==============================] - 258s 475ms/step - loss: 17.5459 - crf_accuracy: 0.8127 - val_loss: 17.1855 - val_crf_accuracy: 0.8262
Epoch 12/100
544/544 [==============================] - 240s 442ms/s

Train on 544 samples, validate on 136 samples
Epoch 1/100
128/544 [======>.......................] - ETA: 5:34 - loss: 19.1040 - crf_accuracy: 0.1232

KeyboardInterrupt: 

# ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

## IF a word has a uppercase character ，it’s very likely a skill word

## change network structure ,add a addtional input layer
## change dataset if word contains uppercase label 1， if word are lowercase label 0
## test result

In [16]:
def PreProcessInputData_Addtion( text ):
    word_labels = []
    seq_types = []
    upper_list = []
    

    for sequence in text:
        len_text = len(sequence)

        ###########################################
        temp_word_labels = []
        temp_upper_list = []
        
        temp_word_labels.append( 101 )
        temp_upper_list.append(0)
        for w in sequence:
            temp_word_labels.append( vocab.get(str(w).lower(),100) )
            if str(w).isalpha() and (str(w).islower() == False):
                temp_upper_list.append( 2 )
            else:
                temp_upper_list.append( 1 )
            
        temp_word_labels.append( 102 )
        temp_upper_list.append(0)
        ###########################################
        ###########################################
        temp_seq_types = [1] * len(temp_word_labels) +  [0] * ( max_seq_length - len( temp_word_labels ))
        temp_word_labels = temp_word_labels + [0] * ( max_seq_length - len( temp_word_labels ))
        temp_upper_list  = temp_upper_list  + [0] * ( max_seq_length - len( temp_upper_list ))

        word_labels.append( temp_word_labels )
        seq_types.append( temp_seq_types )
        upper_list.append( temp_upper_list )

    return word_labels, seq_types, upper_list

In [17]:
def PreProcessOutputData_Addtion( text ):
    tags = []
    for line in text:
        tag = [0]
        for item in line:
            tag.append( int(label[item.strip()]) )
        tag.append(0)
        tags.append(tag)

    pad_tags = pad_sequences(tags, maxlen=max_seq_length, padding="post", truncating="post")
    result_tags = np.expand_dims(pad_tags, 2)
    return result_tags

In [18]:
train_sentence_list_word_labels ,train_sentence_list_seq_types , train_sentence_upper_list = PreProcessInputData_Addtion( X_train )
train_output_label_list = PreProcessOutputData_Addtion( y_train )

In [19]:
test_sentence_list_word_labels ,test_sentence_list_seq_types , test_sentence_upper_list = PreProcessInputData_Addtion( X_test )
test_output_label_list = PreProcessOutputData_Addtion( y_test )

In [20]:
def Train_Model_Addtion( network_struct_dict ):
    print( '■' * 60 )
    print('network struct:')
    pprint( network_struct_dict )
    
    #################################################################
    ## use BERT embedding layer
    model_path = r"E:/Project/uncased_L-2_H-128_A-2/"
    bert = load_trained_model_from_checkpoint(
        model_path + "bert_config.json",
        model_path + "bert_model.ckpt",
        seq_len = max_seq_length
        )

    #make bert layer trainable
    for layer in bert.layers:
        layer.trainable = network_struct_dict['is_embeding_trainable']
        
    x1 = Input(shape=(None,))
    x2 = Input(shape=(None,))
    x3 = Input(shape=(None,))
    
    bert_out = bert([x1, x2])
    label_layer1 = Embedding(4, 2, trainable=False)(x3)

    x4 = Concatenate()( [bert_out, label_layer1] )
    print(x4.shape)
    
    my_output = x4
    #################################################################
    # use layer_dict_List to build model
    for layer_dict in network_struct_dict['layer_dict_List']:
        my_input  = my_output
        my_output = define_Layer( layer_dict, my_input )
    
    
    #################################################################
    ## CRF output
    crf_out = CRF(len(label), sparse_target=True)(my_output)
    model = Model([x1, x2, x3], crf_out)
    model.summary()

    model.compile(
        optimizer=network_struct_dict['optimizer'],
        loss=crf_loss,
        metrics=[crf_accuracy]
    )
    
    #################################################################
    ## training
    model.fit( [train_sentence_list_word_labels, train_sentence_list_seq_types, train_sentence_upper_list],
                       y = train_output_label_list,
                       batch_size = network_struct_dict['batch_size'],
                       epochs = network_struct_dict['epochs'],
                       validation_split=0.2,
                       callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001)],
                       verbose = 1,
                       class_weight = 'auto')
    
    print('evaluate :')
    res = model.evaluate( [test_sentence_list_word_labels, test_sentence_list_seq_types, test_sentence_upper_list],
                       y=test_output_label_list,batch_size=128)
    
    print( res )
    return res[1]

In [21]:
network_struct_dict = {
    'is_embeding_trainable' : True,
    'optimizer' : Adam(1e-4),
    'batch_size' : 128,
    'epochs': 100,

    'layer_dict_List' : [ 
        {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
#         {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
    ]
}

In [22]:
Train_Model_Addtion( network_struct_dict )

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
network struct:
{'batch_size': 128,
 'epochs': 100,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 128}],
 'optimizer': <keras.optimizers.Adam object at 0x00000000190BC7B8>}




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







(?, ?, 130)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
______________________________________________________________________________

544/544 [==============================] - 18s 32ms/step - loss: 17.3129 - crf_accuracy: 0.8176 - val_loss: 16.9061 - val_crf_accuracy: 0.8457
Epoch 15/100
544/544 [==============================] - 18s 33ms/step - loss: 17.2781 - crf_accuracy: 0.8271 - val_loss: 16.8764 - val_crf_accuracy: 0.8548
Epoch 16/100
544/544 [==============================] - 18s 33ms/step - loss: 17.2410 - crf_accuracy: 0.8368 - val_loss: 16.8501 - val_crf_accuracy: 0.8684
Epoch 17/100
544/544 [==============================] - 18s 33ms/step - loss: 17.2068 - crf_accuracy: 0.8498 - val_loss: 16.8272 - val_crf_accuracy: 0.8750
Epoch 18/100
544/544 [==============================] - 18s 33ms/step - loss: 17.1836 - crf_accuracy: 0.8593 - val_loss: 16.8055 - val_crf_accuracy: 0.8807
Epoch 19/100
544/544 [==============================] - 18s 33ms/step - loss: 17.1545 - crf_accuracy: 0.8661 - val_loss: 16.7882 - val_crf_accuracy: 0.8851
Epoch 20/100
544/544 [==============================] - 18s 33ms/step - loss:

0.9080848892529806

In [128]:
import random

ranIndex = random.randint(0,len(X_train))
print( X_train[ranIndex] )
print( train_sentence_upper_list[ranIndex][:len(X_train[ranIndex]) + 1] )

['Knowledge', 'of', 'Hadoop', '(', 'HDFS', ',', 'Sqoop', ',', 'Pig', ',', 'Hive', '…', ')', 'preferred']
[0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1]
